In [33]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import os
import numpy as np

Get the Data

In [10]:
base_dir = '/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/'
index_values_file = 'nifty50_index_prices_2021_2025.csv'
index_cons_file = 'nifty50_cons.csv'

In [31]:
index_values = pd.read_csv(base_dir + index_values_file, usecols=['Date', 'Adj Close'], parse_dates=['Date'])
index_values['Index_daily_return'] = index_values['Adj Close'].pct_change()
index_values.dropna(inplace=True)
index_values

,Date,Adj Close,Index_daily_return
1,2021-01-04,14132.900391,0.008161
2,2021-01-05,14199.500000,0.004712
3,2021-01-06,14146.250000,-0.003750
4,2021-01-07,14137.349609,-0.000629
5,2021-01-08,14347.250000,0.014847
...,...,...,...
1230,2025-12-23,26177.150391,0.000181
1231,2025-12-24,26142.099609,-0.001339
1232,2025-12-26,26042.300781,-0.003818
1233,2025-12-29,25942.099609,-0.003848


In [46]:
index_cons = pd.read_csv(base_dir + index_cons_file, parse_dates=['Date'], usecols=['Date', 'Security Symbol'])
index_cons.insert(1, 'Index_name', 'Nifty 50')
index_cons

,Date,Index_name,Security Symbol
0,2021-03-31,Nifty 50,ADANIPORTS
1,2021-03-31,Nifty 50,JSWSTEEL
2,2021-03-31,Nifty 50,KOTAKBANK
3,2021-03-31,Nifty 50,LT
4,2021-03-31,Nifty 50,M&M
...,...,...,...
495,2025-09-30,Nifty 50,HINDALCO
496,2025-09-30,Nifty 50,HINDUNILVR
497,2025-09-30,Nifty 50,ICICIBANK
498,2025-09-30,Nifty 50,COALINDIA


manually remove HDFC, ZOMATO as they have data under different ticker name

In [47]:
index_cons = index_cons[~index_cons['Security Symbol'].isin(['HDFC', 'ZOMATO'])]
index_cons

,Date,Index_name,Security Symbol
0,2021-03-31,Nifty 50,ADANIPORTS
1,2021-03-31,Nifty 50,JSWSTEEL
2,2021-03-31,Nifty 50,KOTAKBANK
3,2021-03-31,Nifty 50,LT
4,2021-03-31,Nifty 50,M&M
...,...,...,...
495,2025-09-30,Nifty 50,HINDALCO
496,2025-09-30,Nifty 50,HINDUNILVR
497,2025-09-30,Nifty 50,ICICIBANK
498,2025-09-30,Nifty 50,COALINDIA


In [48]:
index_cons['Security Symbol'] = index_cons['Security Symbol'].replace('TATAMOTORS', 'TMCV')
index_cons

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_24247/1888627043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_cons['Security Symbol'] = index_cons['Security Symbol'].replace('TATAMOTORS', 'TMCV')


,Date,Index_name,Security Symbol
0,2021-03-31,Nifty 50,ADANIPORTS
1,2021-03-31,Nifty 50,JSWSTEEL
2,2021-03-31,Nifty 50,KOTAKBANK
3,2021-03-31,Nifty 50,LT
4,2021-03-31,Nifty 50,M&M
...,...,...,...
495,2025-09-30,Nifty 50,HINDALCO
496,2025-09-30,Nifty 50,HINDUNILVR
497,2025-09-30,Nifty 50,ICICIBANK
498,2025-09-30,Nifty 50,COALINDIA


In [25]:
price_data = pd.read_csv(base_dir + 'all_price_data.csv', parse_dates=['Date'])
price_data

,Symbol,Date,Close,Volume
0,BHARTIARTL,2020-01-01,433.316467,5154996
1,BHARTIARTL,2020-01-02,435.132721,4933053
2,BHARTIARTL,2020-01-03,435.037109,5154587
3,BHARTIARTL,2020-01-06,429.827362,7538915
4,BHARTIARTL,2020-01-07,425.477997,4353883
...,...,...,...,...
84852,M&M,2025-12-24,3636.699951,842897
84853,M&M,2025-12-25,3636.699951,0
84854,M&M,2025-12-26,3623.100098,829910
84855,M&M,2025-12-29,3592.100098,1034692


In [30]:
price_data['daily_return'] = price_data.groupby('Symbol')['Close'].pct_change()
price_data.dropna(inplace=True)
price_data

,Symbol,Date,Close,Volume,daily_return
1,BHARTIARTL,2020-01-02,435.132721,4933053,0.004192
2,BHARTIARTL,2020-01-03,435.037109,5154587,-0.000220
3,BHARTIARTL,2020-01-06,429.827362,7538915,-0.011975
4,BHARTIARTL,2020-01-07,425.477997,4353883,-0.010119
5,BHARTIARTL,2020-01-08,438.621796,14118818,0.030892
...,...,...,...,...,...
84852,M&M,2025-12-24,3636.699951,842897,0.003200
84853,M&M,2025-12-25,3636.699951,0,0.000000
84854,M&M,2025-12-26,3623.100098,829910,-0.003740
84855,M&M,2025-12-29,3592.100098,1034692,-0.008556


Calculate Beta and volatitlty of each constituent on index cons

In [49]:
def one_year_beta(symbol, value_date):
    start_date = pd.to_datetime(value_date) - pd.DateOffset(years=1)
    end_date = pd.to_datetime(value_date)
    
    stock_data = price_data[(price_data['Symbol'] == symbol) & 
                            (price_data['Date'] >= start_date) & 
                            (price_data['Date'] <= end_date)]
    index_data = index_values[(index_values['Date'] >= start_date) & 
                             (index_values['Date'] <= end_date)]
    
    if stock_data.empty or index_data.empty:
        return np.nan  # Return NaN if data is insufficient
    
    merged_data = pd.merge(stock_data, index_data, left_on='Date', right_on='Date', suffixes=('_stock', '_index'))
    if merged_data.empty:
        return np.nan  # Return NaN if no merged data
    
    X = merged_data['Index_daily_return']
    y = merged_data['daily_return']
    
    if len(X) < 2:  # Ensure enough data points for regression
        return np.nan
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model.params['Index_daily_return']

# Apply the function with error handling
index_cons['Beta'] = index_cons.apply(
    lambda row: one_year_beta(row['Security Symbol'], row['Date']), axis=1
)

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_24247/1148718232.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_cons['Beta'] = index_cons.apply(


In [50]:
def one_year_volatility(symbol, value_date):
    start_date = pd.to_datetime(value_date) - pd.DateOffset(years=1)
    end_date = pd.to_datetime(value_date)

    stock_data = price_data[(price_data['Symbol'] == symbol) & 
                            (price_data['Date'] >= start_date) & 
                            (price_data['Date'] <= end_date)]

    if stock_data.empty:
        return np.nan  # Return NaN if data is insufficient

    # Calculate daily returns
    daily_returns = stock_data['daily_return']

    if len(daily_returns) < 2:  # Ensure enough data points for calculation
        return np.nan

    # Calculate annualized volatility
    volatility = np.std(daily_returns) * np.sqrt(252)  # Assuming 252 trading days in a year
    return volatility
# Apply the function with error handling
index_cons['Volatility'] = index_cons.apply(
    lambda row: one_year_volatility(row['Security Symbol'], row['Date']), axis=1) 

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_24247/84915883.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_cons['Volatility'] = index_cons.apply(


Delete the tickers with missing beta or volatilty due to insufficient data

In [51]:
index_cons = index_cons.dropna(subset=['Beta', 'Volatility'])
index_cons

,Date,Index_name,Security Symbol,Beta,Volatility
0,2021-03-31,Nifty 50,ADANIPORTS,1.048103,0.363451
1,2021-03-31,Nifty 50,JSWSTEEL,1.007288,0.407596
2,2021-03-31,Nifty 50,KOTAKBANK,0.984397,0.387015
3,2021-03-31,Nifty 50,LT,1.151274,0.328698
4,2021-03-31,Nifty 50,M&M,1.227371,0.453830
...,...,...,...,...,...
495,2025-09-30,Nifty 50,HINDALCO,1.216176,0.277803
496,2025-09-30,Nifty 50,HINDUNILVR,0.441855,0.195020
497,2025-09-30,Nifty 50,ICICIBANK,0.880423,0.173273
498,2025-09-30,Nifty 50,COALINDIA,0.933042,0.213540


In [52]:
index_cons.to_csv(os.path.join(base_dir, 'nifty50_cons_with_risk_metrics.csv'), index=False)